In [ ]:
from dotenv import load_dotenv

load_dotenv()

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# chain = (
#         {
#             "context": retriver,
#             "question": RunnablePassthrough(),
#             "history": memory.load_memory_variables({})["history"]
#         }
#         | prompt
#         | llm
# )
#
# chain.invoke("Describe Victory Mansions")


# def load_memory(_):
#     return memory.load_memory_variables({})["history"]


# chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm
chain = prompt | llm


def invoke_chain(question):
    result = chain.invoke(
        {
            "context": retriever,
            "question": question,
            # FIXME
            "history": memory.load_memory_variables({})["history"]
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

In [10]:
invoke_chain("Aaronson 은 유죄인가요?")

content='제가 알기로는 Aaronson에 대한 정보는 제가 가지고 있지 않습니다.'


In [11]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

content='제가 알기로는 그에 대한 정보는 제가 가지고 있지 않습니다.'


In [12]:
invoke_chain("Julia 는 누구인가요?")

content='제가 알기로는 Julia는 FAISS와 CacheBackedEmbeddings와 관련이 있는 것으로 알고 있습니다.'
